## Synthetize multiple audio 
+ Make a command for synthesize audio file from text
+ Synthetize audio
+ Synthetize multiple audio

In [1]:
def counter(num=1, length=3):
    """Counter etc. 0001, 0002
    Attributes:
    num (int) integer etc. 1 ==> 0001
        length (int) length of counter etc. 3 ==> 001
    Return:
        (str) etc. 0001
    """
    number = '0' * length + str(num)
    number = number[len(number)-length:]
    return number

### Make a command

In [2]:
import re


def make_command(text, name, save, quality):    
    return re.sub(" +", " ", f"""
    source .venv/bin/activate && \
    python main.py \
        -t "{text}" \
        -n "{name}" \
        -q "{quality}" \
        -s "{save}"
    """).strip()

text = "Hello World"
name = "kathleen-glow_tts"
save = "/root/Desktop/sample.wav"
quality = "low"

cmd = make_command(text, name, save, quality)
cmd

'source .venv/bin/activate && python main.py -t "Hello World" -n "kathleen-glow_tts" -q "low" -s "/root/Desktop/sample.wav"'

### Synthetize audio

In [3]:
import subprocess as sb

def synthesize_audio(cmd, cwd):
    process = sb.Popen(
        cmd, cwd=cwd, shell=True, 
        executable='/bin/bash', encoding='UTF-8',
        stdin=sb.PIPE, stdout=sb.PIPE, stderr=sb.PIPE
    )
    stdout, stderr = process.communicate('')

cwd = "/root/PycharmProjects/Pytorch/03 - Larynx TTS"

# synthesize_audio(cmd, cwd)

### Synthetize multiple audio

In [4]:
def load_text(meta):
    with open(meta, "r", encoding="utf-8") as file:
        texts = [line.split('|')[-1] for line in file.read().splitlines()]
    return texts

meta = "/root/PycharmProjects/Tensorflow-2.X/07 - DeepSpeech Vad Transcriber + Create Wavs/text.txt"

load_text(meta)

['Great party, isn’t it?',
 'Yeah. It’s cool.',
 'I’m Ruth.',
 'I’m Julia. Nice to meet you.',
 'You too.',
 'Hi, I’m Kevin.',
 'Hi, Kevin! I’m David. Nice to meet you.',
 'You too, David. So, how do you know the host?',
 'I and Daniel work together. What about you?',
 'I live next door.',
 'Laura, meet Seon! Seon, this is Laura!',
 'Nice to meet you, Seon.',
 'You too, Laura!',
 'James, this is Chris. Chris, this is James.',
 'Nice to meet you, James!',
 'You too, Chris. I’ve heard a lot about you!',
 'All good, I hope.',
 'Hi, I’m Claire.',
 'Nice to meet you. I’m Robert.',
 'You too. Are you new here?',
 'Yes, it’s my first day here.',
 'You must be Susan.',
 'Yes, right.',
 'I’m Jill. Nice to meet you, Susan. Let me show you around.',
 'Nice to meet you too, Jill. Thank you.',
 'Hello, Jake. Welcome to the Nesco Company. Let me introduce you to some people.',
 'Thank you, that would be nice.',
 'Hi, I’m Jeff. Are you new here?',
 'Yes, I’m Alex. I’m a new paralegal.',
 'Where will 

### Transcribe audio

In [5]:
import deepspeech
import pydub

class DeepSpeech:
    def __init__(self, model, scorer, alpha=0.931289039105002, beta=1.1834137581510284):
        self.model = deepspeech.Model(model)
        self.model.enableExternalScorer(scorer)
        self.model.setScorerAlphaBeta(alpha, beta)
   
    def transcribe(self, audio):
        audio_segment = pydub.AudioSegment.from_wav(audio)
        audio_segment = audio_segment.set_frame_rate(16000)
        assert audio_segment.frame_rate == 16000 
        samples = audio_segment.get_array_of_samples()
        return self.model.stt(samples)

    
#model_path = 'model/output_graph.tflite'
#scorer_path = 'model/output_graph.scorer'
#audio_path = "model/sample.wav"

#model = DeepSpeech(model_path, scorer_path)
#model.transcribe(audio_path)

In [ ]:
import os
import re

from nltk.translate.bleu_score import sentence_bleu

# FUNCTIONS
def compute_bleu(candidate, reference, gram=1):
    candidate = candidate.split(" ")
    reference = [reference.split(" ")]
    weights = [0, 0, 0, 0]
    weights[gram - 1] = 1 
    return sentence_bleu(reference, candidate, weights=weights) 


def make_command(text, name, save, quality):    
    return re.sub(" +", " ", f"""
    source .venv/bin/activate && \
    python main.py \
        -t "{text}" \
        -n "{name}" \
        -q "{quality}" \
        -s "{save}"
    """).strip()

def filter_text(text):
    # Filter chars [a-z' ]
    text = text.lower()
    text = re.sub(r"’m", r"'m", text)
    text = re.sub(r"’s", r"'s", text)
    text = re.sub(r"’re", r"'re", text)
    text = re.sub(r"’ve", r"'ve", text)
    text = re.sub(r"[^\w ']", "", text)     # a-z'
    text = re.sub(" +", " ", text)          # "  " -> " "
    return text.strip()


# PATHS
base = "/".join(os.path.abspath(os.path.dirname(__name__)).split("/")[:-1])
model_path = f'{base}/model/output_graph.tflite'
scorer_path = f'{base}/model/output_graph.scorer'

names = f"/root/PycharmProjects/Pytorch/03 - Larynx TTS/voices/en-us"
text_path = "/root/PycharmProjects/Tensorflow-2.X/07 - DeepSpeech Vad Transcriber + Create Wavs/text.txt"
cwd = "/root/PycharmProjects/Pytorch/03 - Larynx TTS"
save = "/root/Desktop/wavs"
quality = "high"
meta = "/root/Desktop/metadata.csv"

# DIRECTORIES
os.makedirs(save, exist_ok=True)

# VARIABLES
count = 1
model = DeepSpeech(model_path, scorer_path)

with open(meta, "w", encoding="utf-8") as file:    
    for text in load_text(text_path):
        for speaker in os.listdir(names):
            name = f"SA-001-{counter(count, 4)}"
            audio_path = f"{save}/{name}.wav"
            cmd = make_command(text, speaker, audio_path, quality)
            synthesize_audio(cmd, cwd)            
            transcript = model.transcribe(audio_path)
            
            text = filter_text(text)
            bleu = compute_bleu(transcript, text, gram=1)
            print(transcript, text, bleu)
            print()
            file.writelines(f"{name}|{text}|{transcript}|{bleu}\n")
            count += 1
            
load_text(meta)

TensorFlow: v2.3.0-6-g23ad988
DeepSpeech: v0.9.3-0-gf2e9c85
/root/anaconda3/envs/DeepSpeech/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/root/anaconda3/envs/DeepSpeech/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


great party isn't it great party isnt it 0.75

great party is it great party isnt it 0.75

great party is to great party isnt it 0.5

great party is  great party isnt it 0.5



/root/anaconda3/envs/DeepSpeech/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


great parties it great party isnt it 0.47768754038252614

gay parties to great party isnt it 0

great party isn't it great party isnt it 0.75

great parties it great party isnt it 0.47768754038252614

great party isn't it great party isnt it 0.75

great party is it great party isnt it 0.75

great parties it great party isnt it 0.47768754038252614

great parties et great party isnt it 0.23884377019126307

great party isn't it great party isnt it 0.75

great parties and that great party isnt it 0.25

great party as it great party isnt it 0.75

great party is in it great party isnt it 0.6

great parties it great party isnt it 0.47768754038252614

great parties and to  great party isnt it 0.2

great party isn't it great party isnt it 0.75

great parkes  great party isnt it 0.23884377019126307

great party isn't it great party isnt it 0.75

great party is it great party isnt it 0.75

great parties into great party isnt it 0.23884377019126307

great parties tit great party isnt it 0.23884377

i have an i'm david nice to meet you hi kevin i'm david nice to meet you 0.6666666666666666

hi kevin i'm david nice to meet you hi kevin i'm david nice to meet you 1.0

i even in david nice to meet you hi kevin i'm david nice to meet you 0.625

hi kevin and david nice to me you hi kevin i'm david nice to meet you 0.75

e two dated so how do you know the host you too david so how do you know the host 0.7

you two david so how do you know the host you too david so how do you know the host 0.9

your two david so how do you know the host you too david so how do you know the host 0.8

you to day bid so how do you know the host you too david so how do you know the host 0.7272727272727273

you two david so how do you know the host you too david so how do you know the host 0.9

i to david so how do you know the house you too david so how do you know the host 0.7

you two david so how do you know the host you too david so how do you know the host 0.9

you to day it so how do you know the house

...

would you like to come to my birthday party would you like to come to my birthday party 1.0

would you like to come to my birthday party would you like to come to my birthday party 1.0

would you like to come to my birthday party would you like to come to my birthday party 1.0

would you like to come to my bar day parke would you like to come to my birthday party 0.7

would you like to come to my burdet party would you like to come to my birthday party 0.8888888888888888

where you like to come to my birth a party would you like to come to my birthday party 0.7

would you like to come to my birthday party would you like to come to my birthday party 1.0

would you like to come to my bat to party would you like to come to my birthday party 0.8

would you like to come to my birth at party would you like to come to my birthday party 0.8

would you like to come to my birthday party would you like to come to my birthday party 1.0

would you like to come to my birthday party would you like to

that would be lovely but i've got a previous arrangement that would be lovely but i've got a previous arrangement 1.0

that would be lovely but i've got a previous arrangement that would be lovely but i've got a previous arrangement 1.0

that would be lovely but i've got a previous arrangement that would be lovely but i've got a previous arrangement 1.0

that would be lovely but i toss a previous arrangement that would be lovely but i've got a previous arrangement 0.8

that would be lovely but i've got a previous arrangement that would be lovely but i've got a previous arrangement 1.0

that would be lovely but i've got a previous arrangement that would be lovely but i've got a previous arrangement 1.0

that would be lovely but i got a perisher intent that would be lovely but i've got a previous arrangement 0.7

these would be lovely but i've got a previous rangement that would be lovely but i've got a previous arrangement 0.8

that would be lovely but i'd go a previous arrangement that

i'd love it if you could come to state what do you mind dog id love it if you could come to stay but do you mind dogs 0.7142857142857143

i'd love it if you could come to stay but to you mine dogs id love it if you could come to stay but do you mind dogs 0.7857142857142857

i'd love it if you could come to state to you in doors id love it if you could come to stay but do you mind dogs 0.5698222022414253

it love it if you could come to state but do you mind dogs id love it if you could come to stay but do you mind dogs 0.8571428571428571

i'd love it if you could come to state the do you mind dogs id love it if you could come to stay but do you mind dogs 0.7857142857142857

i'd love it if you could come to state what do you mine old id love it if you could come to stay but do you mind dogs 0.6428571428571429

i'd love it if you could come to state bade you in dogs id love it if you could come to stay but do you mind dogs 0.6410499775216034

no of course not thank you for inviting me no

thank you for asking me thank you for asking me 1.0

thank you for asking me thank you for asking me 1.0

thank you for using me thank you for asking me 0.8

thank you for asking me thank you for asking me 1.0

thank you for asking me thank you for asking me 1.0

thank you for asking me thank you for asking me 1.0

thank you for asking me thank you for asking me 1.0

thank you for asking me thank you for asking me 1.0

thank you for asking me thank you for asking me 1.0

thank you for asking me thank you for asking me 1.0

i come in hi come in 0.6666666666666666

i come in hi come in 0.6666666666666666

i come in hi come in 0.6666666666666666

i come in hi come in 0.6666666666666666

i come in hi come in 0.6666666666666666

i come in hi come in 0.6666666666666666

i come on hi come in 0.3333333333333333

i come in hi come in 0.6666666666666666

i come in hi come in 0.6666666666666666

i come in hi come in 0.6666666666666666

i come on hi come in 0.3333333333333333

i come on hi come in

nice to have you here nice to have you here 1.0

my ter have you here nice to have you here 0.6

nice to have you here nice to have you here 1.0

nice to have you here nice to have you here 1.0

nice to have you here nice to have you here 1.0

nice to have you here nice to have you here 1.0

nice to have you here nice to have you here 1.0

nice to have you here nice to have you here 1.0

nice to have you here nice to have you here 1.0

not to have you here nice to have you here 0.8

not to have you here nice to have you here 0.8

nice to have you here nice to have you here 1.0

nice to have you here nice to have you here 1.0

not a half year nice to have you here 0

nice to have you here nice to have you here 1.0

i'm glad i could come i'm glad i could come 1.0

i'm glad i could come i'm glad i could come 1.0

i'm glad i could come i'm glad i could come 1.0

i'm glad i could come i'm glad i could come 1.0

i'm glad i could come i'm glad i could come 1.0

i'm glad i could come i'm glad 